# Using `Gmsh`

In this demo we will show how to generate Purkinje networks on an idealized bi-ventricular geometry generated with gmsh. In order to create this geometry with will use another library called [`cardiac-geometries`](https://computationalphysiology.github.io/cardiac_geometries/) which can be installed with pip, i.e
```
pip install cardiac-geometries
```
Note also that to create this geometry you need `gmsh` installed with `OpenCASACADE`. Normally this will come with gmsh if you install it from [pypi](https://pypi.org/project/gmsh/#files) but if you happen to be on a Mac M1, then you need to build this from source. In this case it might be easier to use the [pre-built docker](https://github.com/scientificcomputing/packages/pkgs/container/fenics-gmsh) container that we have made.

First we will import the necessary libraries

In [ ]:
import numpy as np
import logging
from pathlib import Path
import meshio  # pip install meshio
from fractal_tree import generate_fractal_tree, FractalTreeParameters, Mesh

And set the logging level to INFO

In [ ]:
logging.basicConfig(level=logging.INFO)

We will also set a seed for the random number generator so that the results we generate are reproducible.

In [ ]:
np.random.seed(1234)

The mesh that we create from `gmsh` is not directly possible to visualize in Paraview, so in order to do that we create a simple helper function for converting the mesh to `XDMF` format which is possible to visualize in Paraview.

In [ ]:
def create_mesh(mesh, cell_type):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:geometrical", cell_type)
    out_mesh = meshio.Mesh(
        points=mesh.points,
        cells={cell_type: cells},
        cell_data={"name_to_read": [cell_data]},
    )

    return out_mesh

We will now create the mesh. If the mesh already exist we skip this step. Here we need to import `cardiac-geometries` and call the `biv_ellipsoid` function from the `gmsh` sub-package. We alse set the characteristic length to 0.1 which will generate a finer mesh that what is the default.

In [ ]:
path = Path("biv_ellipsoid.msh")
if not path.is_file():
    # Generate mesh with cardiac geometries
    import cardiac_geometries

    cardiac_geometries.gmsh.biv_ellipsoid(path, char_length=0.1)

The generated mesh will be saved to the given path. We can now load this mesh using `meshio`.

In [ ]:
msh = meshio.read(path)

For pure visualization purposes we will also save the mesh in `.xdmf` format so that we can visualize the geometry in together with the Purkinje networks in the end.

In [ ]:
tetra_mesh = create_mesh(msh, "tetra")
meshio.write(path.with_suffix(".xdmf"), tetra_mesh)

For this example we will generate the Purkinje networks on the left ventricular endocardium. This mesh comes with some physical surfaces with different tags. These tags are stored in the `field_ data` attribute

In [ ]:
print(msh.field_data)

We will use the tag for the `ENDO_LV`

In [ ]:
tag = msh.field_data["ENDO_LV"][0]

We also need to extract the relevant cells with this tag

In [ ]:
inds = [i for i, x in enumerate(msh.cell_data["gmsh:physical"]) if x[0] == tag]
connectivity = np.vstack([msh.cells[i].data for i in inds])

The next task is to define the starting point for the network. We would like this to be a point close to the base of the left ventricular endocardial septum. You could use Paraview to find approximately what the coordinate of this point should be. In this case a suitable choice for the initial node would the $(0, 1, 0)$.

In [ ]:
init_node = [0, 1, 0]

Now that we have the approximate coordinate, we need to find the closest node in the mesh. We can do this by selecting the point with the shortest distance to this node.

In [ ]:
verts = msh.points
index = np.linalg.norm(np.subtract(verts, init_node), axis=1).argmin()

We create the mesh, and pass in the initial node

In [ ]:
mesh = Mesh(verts=verts, connectivity=connectivity, init_node=verts[index, :])

We also set the number of generations to 15 and we set the initial direction to point in the positive $x$-direction which is the direction from the base towards the apex (this can also be found by visualizing the mesh in Paraview)


In [ ]:
param = FractalTreeParameters(
    filename="biv-line-lv", N_it=15, initial_direction=np.array([1, 0, 0])
)

Next we create the Purkinje networks

In [ ]:
branches, nodes = generate_fractal_tree(mesh, param)

```{figure} ../docs/figures/biv_gmsh.jpg
---
name: biv_gmsh
---
Idealized bi-ventricular geometry with generated Purkinje networks on the LV endocardium originating from the basal septum
```